# Machine Learning Final Project - Facial expression recognition

## 1. Project proposal: 

We will train a facial recognition system from scratch which will finally recognize the emotion by  analysing images/videos. Our system will attempt to recognize six basic emotional expressions including fear, disgust, anger, surprise, happiness, and sadness introduced by Ekman[1]. Because the racial effect will occur in the process of facial recognition. We will also try to figure out how to detect races which may reduce the influence of the effect. 
/n    To make this system more interesting, we plan to transform the final result from the plain words into emoji. This means that the output will be an emoji that can represent the emotion and the race of the human.There may be other human-related information we can detect from static images and video sequences, like genders, ages, hair colors and so on. Facial expression and race will be the most important feature that we will focus on.

## 2. Program Implementation:

### 2.1 Preprocessing

#### 2.1.1 get_train_data.py

#### a. tranverse all images

In [ ]:
def get_race_data():
    file_path = '../img/CFDImages/'  # path of the file that stores the raw images
    res = []
    clss = ['A','B','L','W']

    detector = face_feature.get_detector()
    predictor = face_feature.get_predictor()

    for (root, dirs, files) in os.walk(file_path, ):
        for dir in dirs:
            img_root = os.path.join(root, dir)
            for img_name in os.listdir(img_root):
                if img_name.endswith('N.jpg'):
                    full_path = os.path.join(img_root,img_name)
                    print(full_path)
                    _,fts = face_feature.get_all_feature(full_path,detector,predictor)

                    for ft in fts:
                        cls = img_name[4]
                        rj = [clss.index(cls)]
                        for k in ft:
                            rj.append(k)
                        res.append(rj)
                        print(len(res))
    res = np.asarray(res)

    save_path = '../img/CFDImages/'
    if save_feature(save_path, res):
        print("feature saved!")
    return res

#### b. save features into feature.npy file, and format as below:
    | label | features |
    |   0   |   ...    |
    |   1   |   ...    |
    |  ...  |   ...    |
            
    label : multiclass {0,1,...};
    features : get_face_feature in face_feature.py

In [ ]:
def save_feature(path, res):
    try:
        np.save(path+'feature', res)
        return 1
    except:
        print("Error saving res")
        return 0

def save_race_feature(path, res):
    try:
        np.save(path+'race', res)
        return 1
    except:
        print("Error saving res")
        return 0

#### c. extract race features and save to into file
get racial features from get_all_feature()

In [ ]:
def get_race_data():
    file_path = '../img/CFDImages/'  # path of the file that stores the raw images
    res = []
    clss = ['A','B','L','W']

    detector = face_feature.get_detector()
    predictor = face_feature.get_predictor()

    for (root, dirs, files) in os.walk(file_path, ):
        for dir in dirs:
            img_root = os.path.join(root, dir)
            for img_name in os.listdir(img_root):
                if img_name.endswith('N.jpg'):
                    full_path = os.path.join(img_root,img_name)
                    print(full_path)
                    _,fts = face_feature.get_all_feature(full_path,detector,predictor)

                    for ft in fts:
                        cls = img_name[4]
                        rj = [clss.index(cls)]
                        for k in ft:
                            rj.append(k)
                        res.append(rj)
                        print(len(res))
    res = np.asarray(res)

    save_path = '../img/CFDImages/'
    if save_feature(save_path, res):
        print("feature saved!")
    return res

### 2.1.2 face_feature.py

#### a. get all features from path or camera;

In [ ]:
def all_face_feature(path):
    detector = get_detector()
    predictor = get_predictor()
    return get_all_feature(path,detector,predictor)

In [ ]:
def get_all_feature_from_camera(img, detector, predictor):
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    faces = detector(gray_img, 0)

    res_landmark = []
    res_race = []
    # get features
    for (i, face) in enumerate(faces):
        shape = predictor(gray_img, face)
        fts = face_utils.shape_to_np(shape)

        # square the faces and normalize the feature
        norm = np.amax(fts, 0) - np.amin(fts, 0)
        fts_norm = (fts - np.amin(fts, 0)) / norm
        fts_ret = fts_norm.reshape((1, fts_norm.shape[0] * fts_norm.shape[1]), order='A').tolist()[0]
        res_landmark.append(fts_ret)

        # get skin color feature
        std_landmark = [[37,38,42],[38,39,42],[38,39,41],[39,40,41],[2,29,30],[2,30,31],[2,3,32],[3,4,32]]
        res_hsv = []
        for tri in std_landmark:
            # find a list of points that located inside the triangle
            pts = get_pts_in_tri(fts, tri)

            # find average hsv color space of points inside triangles
            hsv = get_hsv_in_tri(img, pts)

            for i in hsv:
                res_hsv.append(i)

        res_race.append(res_hsv)
    return res_landmark, res_race

#### b. The fuction get_detector() and get_predictor() are dependent on dlib library, defined as below:

In [ ]:
def get_detector():
    detector = dlib.get_frontal_face_detector()
    return detector

In [ ]:
def get_predictor():
    p = "../img/face_landmarks.dat"
    predictor = dlib.shape_predictor(p)
    return predictor

#### c. help function used on reading images from path:

In [ ]:
def get_all_feature(path,detector,predictor):
    img = cv2.imread(path)
    return get_all_feature_from_camera(img, detector, predictor)

#### d. help function used on reading images from camera:

In [ ]:
def get_face_feature(path, detector, predictor):
    # get the image
    img = cv2.imread(path)
    res = get_face_feature_from_camera(img, detector, predictor)
    return res

#### e. set a landmark triangle from face points
<img src="https://ibug.doc.ic.ac.uk/media/uploads/images/annotpics/figure_68_markup.jpg" title="facefeatures" width="500" height="500" />

In [ ]:
std_landmark = [[37,38,42],[38,39,42],[38,39,41],[39,40,41],[2,29,30],[2,30,31],[2,3,32],[3,4,32]]

#### f. extract racial features using hsv
Define three fuctions to get **HSV** feature of skin colors
*deter_in_tri, get_pts_in_tri, get_hsv_in_tri*

> number of feature landmarks: 
>    - expression: 68 x 2 = 136; 
>    - race: 3 x 8 = 24.

In [ ]:
def deter_in_tri(x,y,vers):
    # determine if the point is in the triangle with three vertexes in vers
    # get the result by determine whether the point is on the same side the with each point of the opposite arg
    for i in range(3):
        ab = vers.copy()
        c = ab[i]
        ab.pop(i)
        if S(ab[0],ab[1],c)*S(ab[0],ab[1],[x,y])<0:
            return 0

    return 1

In [ ]:
def get_pts_in_tri(fts,tri):
    pts = []

    vers = np.concatenate((np.asarray([fts[tri[0]-1,:]]),np.asarray([fts[tri[1]-1,:]]),np.asarray([fts[tri[2]-1,:]])),axis = 0)
    xmin = np.min(vers,0)[0]
    xmax = np.max(vers,0)[0]
    ymin = np.min(vers,0)[1]
    ymax = np.max(vers,0)[1]
    vers = vers.tolist()

    # check if pts are in the triangle
    for x in range(xmin,xmax+1):
        for y in range(ymin,ymax+1):
            if deter_in_tri(x,y,vers) == 1:
                pts.append([x,y])

    return pts

In [ ]:
def get_hsv_in_tri(img, pts):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    num = len(pts)
    h = 0
    s = 0
    v = 0

    for pt in pts:
        h = h + hsv_img[pt[0],pt[1],0]
        s = s + hsv_img[pt[0],pt[1],1]
        v = v + hsv_img[pt[0],pt[1],2]

    return [h/num,s/num,v/num]

### 2.1.3 race_feature.py

#### a. extract features of different skin colors from camera capture

In [ ]:
def get_race_feature(path, detector, predictor):
    img = cv2.imread(path)
    res = get_race_feature_from_camera(img, detector, predictor)
    return res

#### b. capture feature from camera frame

In [ ]:
def get_race_feature_from_camera(img, detector, predictor):
    # get feature of the image
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # detect faces
    faces = detector(gray_img, 0)

    res = []
    # get features
    for (i, face) in enumerate(faces):
        shape = predictor(gray_img, face)
        fts = face_utils.shape_to_np(shape)
        # for (x,y) in fts:
        #     cv2.circle(img, (x, y), 2, (0, 255, 0), -1)

        # square the faces and normalize the landmark feature
        norm = np.amax(fts, 0) - np.amin(fts, 0)
        fts_norm = (fts - np.amin(fts, 0)) / norm
        fts_ret = fts_norm.reshape((1, fts_norm.shape[0] * fts_norm.shape[1]), order='A').tolist()[0]
        res.append(fts_ret)

    return res

## 2.2 Training Data

### 2.2.1 get_train_data.py

#### a. get training data from files
Extract features from get_face_feature():

In [ ]:
def get_train_data():
    ...
    fts = face_feature.get_face_feature(full_path, detector, predictor)
    ...

#### b. get_race_data()
Extract features from get_all_features():

In [ ]:
def get_race_data():
    ...
     _,fts = face_feature.get_all_feature(full_path,detector,predictor)
    ...

### 2.2.2 training_data.py

#### a. training data according to the input strategies:

In [ ]:
    def lets_train(x,y,classifier,strategy):
    # decomposition PCA
    pca = decomposition.PCA(n_components=50).fit(x)
    newx = x

    if strategy == 'ova':
        if classifier == 'svm':
            clf = multiclass.OneVsRestClassifier(SVC(C = 1e2, gamma = 'auto')).fit(newx,y)
        elif classifier == 'decisionTree':
            clf = multiclass.OneVsRestClassifier(ensemble.BaggingClassifier(tree.DecisionTreeClassifier(max_depth=1),
                                                                            n_estimators=200)).fit(newx,y)
        elif classifier == 'logistic':
            clf = multiclass.OneVsRestClassifier(LogisticRegression(C=1e8, penalty='l2',solver='newton-cg')).fit(newx,y)
        elif classifier == 'mlp':
            clf = multiclass.OneVsRestClassifier(neural_network.MLPClassifier(solver='sgd',activation = 'relu',
                                                                              max_iter = 100,alpha = 1e-5,hidden_layer_sizes = (300,400,200,200),
                                                                              verbose = True)).fit(newx,y)
        else:
            print("wrong classifier type in ova")
            return 0,1,0

    elif strategy == 'ovo':
        if classifier == 'svm':
            clf = multiclass.OneVsOneClassifier(SVC(C = 1e2, gamma = 'auto')).fit(newx,y)
        elif classifier == 'decisionTree':
            clf = multiclass.OneVsOneClassifier(ensemble.BaggingClassifier(tree.DecisionTreeClassifier(max_depth=1),n_estimators=200)).fit(newx,y)
        elif classifier == 'logistic':
            clf = multiclass.OneVsOneClassifier(LogisticRegression(C=1e8,penalty='l2',solver='newton-cg')).fit(newx,y)
        elif classifier == 'mlp':
            clf = multiclass.OneVsRestClassifier(neural_network.MLPClassifier(solver='sgd',activation = 'logistic',
                                                                              max_iter = 200,alpha = 1e-5,hidden_layer_sizes = (300,300),
                                                                              verbose = True)).fit(newx,y)
        else:
            print("wrong classifier type in ovo")
            return 0, 1, 0

    else:
        print("wrong classifier strategy")
        return 0, 1, 0


    err = np.sum(np.sign(abs(y.T-clf.predict(newx))))/y.shape[0]
    return clf, err, pca

#### b. calculating error rate on testing data according to the classifier and PCA method:

In [ ]:
def lets_test(x,y,clf,pca):
    newx = x
    test_res = clf.predict(newx)
    err = np.sum(np.sign(abs(y.T-test_res)))/test_res.shape[0]
    return test_res,err

#### c. randomly choose 5/6 as training data and rest 1/6 as testing data:

In [ ]:
def select_data(raw_data):
    raw_data = raw_data.tolist()
    n = len(raw_data)

    # get a random sequence:
    seq = np.arange(0,n,1)
    rd.shuffle(seq)

    # put data to each set
    train_data = []
    test_data = []
    for i in range(n):
        if seq[i] < n * 5/6:
            train_data.append(raw_data[i])
        else:
            test_data.append(raw_data[i])

    train_data = np.asarray(train_data)
    test_data = np.asarray(test_data)

    return train_data, test_data

## 2.3 Main function: real time recognition from camera

### 2.3.1 rt_rec.py

In [ ]:
# get trained model
    feature_path = '../img/facesdb/feature.npy'
    strategy = 'ova' # {'ova', 'ovo'}
    classifier = 'svm' # {'svm', 'decisionTree','logistic','mlp'}
    clf, pca = training_data.training_data(feature_path, strategy, classifier)

In [ ]:
 # get real time result
    while(True):
        _,img = cam.read()
        x_offset = 100
        y_offset = 250
        ft = np.asarray(face_feature.get_face_feature_from_camera(img,detector,predictor))

        if ft.shape[0] > 0:
            for i in range(ft.shape[0]):
                res = clf.predict(np.mat(ft[i,:]).A)

                emoji = cv2.imread(emoji_paths[int(res)])
                emoji = cv2.resize(emoji, (int(150), int(150)))

                img = cv2.putText(img, expression[int(res)],(100,200),cv2.FONT_HERSHEY_SIMPLEX,3,(255,255,255),4)
                img[y_offset: y_offset + emoji.shape[0], x_offset: x_offset + emoji.shape[1]] = emoji

        cv2.imshow("camera", img)

        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break

## References:

1.Ekman P., Friesen W.V., Ellsworth P.Emotion in the Human Face: Guide-Lines for Research and an Integration of Findings: Guidelines for Research and an Integration of Findings.Pergamon; Berlin, Germany: 1972.

2.database source: http://app.visgraf.impa.br/database/faces

+ 300-W
+ XM2VTS
+ FRGC Ver.2
+ CFD ver2.0.3: https://chicagofaces.org/
+ C. Sagonas, E. Antonakos, G, Tzimiropoulos, S. Zafeiriou, M. Pantic. 300 faces In-the-wild challenge: Database and results. Image and Vision Computing (IMAVIS), Special Issue on Facial Landmark Localisation "In-The-Wild". 2016.

3.face feature: https://ibug.doc.ic.ac.uk/resources/facial-point-annotations/

+ IMPA-FACE3D(the geometry and texture are correlated)
